In [34]:
import urllib.request as urllib
import cv2
import numpy as np
import time

# Replace the URL with your own IPwebcam shot.jpg IP:port
url='http://192.168.31.149:8080/shot.jpg'

while True:

    # Use urllib to get the image and convert into a cv2 usable format
    imgResp=urllib.urlopen(url)
    imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgNp,-1)
    

    #cv2.rectangle(img, pt1=(100,100), pt2=(200,200), color=(0,255,0), thickness=10)
    img = cv2.filter2D(img, -1, np.ones((3,3))*10)
    
    # put the image on screen
    cv2.imshow('IPWebcam', img)
    
    #To give the processor some less stress
#     time.sleep(1) 

    if cv2.waitKey(20) & 0xFF == 27:
        break
        
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

full = cv2.imread('DATA/sammy.jpg')
full = cv2.cvtColor(full, cv2.COLOR_BGR2RGB)

face = cv2.imread('DATA/target.png')
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
hiegh = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  




import itertools

for n in itertools.count():
    # Use urllib to get the image and convert into a cv2 usable format
    ret, frame = cap.read()

    res = cv2.matchTemplate(frame, face, cv2.TM_CCOEFF_NORMED)

    y_face, x_face, deep= face.shape

    minVal, maxVal, minLoc, maxLoc= cv2.minMaxLoc(res)
    pt1_x, pt1_y =maxLoc
    pt2_x, pt2_y =pt1_x+x_face, pt1_y+y_face 


    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(frame, "max: {}, min: {}".format(maxVal,minVal),org=(0,10),fontFace =font,fontScale=1,color=(0,255,0))
    
    
    if maxVal > 0.5:
        cv2.imwrite('DATA/emoji/smile_{}.png'.format(n), frame[pt1_y:pt2_y,pt1_x:pt2_x])
        cv2.rectangle(frame, pt1 =(pt1_x, pt1_y), pt2=(pt2_x, pt2_y),color=[0,255,0], thickness=5)
        
    
    # put the image on screen
    cv2.imshow('Webcam', frame)

    time.sleep(1/30)
    if cv2.waitKey(1) & 0xFF == 27:
        break

        
        
        
        
plt.subplot(121)
plt.imshow(res)
print(res.shape)
print(face.shape)
print(frame.shape)

plt.subplot(122)
plt.imshow(frame)        
    
    
plt.show()    
cap.release()        
cv2.destroyAllWindows()

# СОЗДАТЬ НАБОР ДАННЫХ

In [15]:
import cv2
import sys, os
import itertools
import time
import numpy as np
import scipy

def smooth(values, threshold = 5):
    #x, y, w, h
    if "to_smooth" not in globals():
        global to_smooth
        to_smooth = np.array([values])
          
    to_smooth = np.concatenate([to_smooth, [values]])  
    
    x = to_smooth[len(to_smooth)-threshold:len(to_smooth),  0].mean()
    y = to_smooth[len(to_smooth)-threshold:len(to_smooth),  1].mean()
    w = to_smooth[len(to_smooth)-threshold:len(to_smooth),  2].mean()
    h = to_smooth[len(to_smooth)-threshold:len(to_smooth),  3].mean()

    to_smooth = to_smooth[len(to_smooth)-threshold:len(to_smooth)]
    return int(x), int(y), int(w), int(h)


def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result




cap   = cv2.VideoCapture(0)
font  = cv2.FONT_HERSHEY_PLAIN

# Get user supplied values
cascPath = r"E:\GitHub\udemy\DATA\haarcascades"
if not os.path.exists(cascPath):
    cascPath = r"D:\GitHub\udemy\DATA\haarcascades"
# Create the haar cascade

eyeCascade = cv2.CascadeClassifier(os.path.join(cascPath, 'haarcascade_eye.xml'))
faceCascade = cv2.CascadeClassifier(os.path.join(cascPath,'haarcascade_frontalface_default.xml'))

path = r'DATA/emoji/idle/'
count_files = len(os.listdir(path))

for n in itertools.count():
    ret, image = cap.read()
    image = cv2.resize(image, (160,120))
    # Read the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _weights = []
    _rects   = []
    _angles  = []
    for angle in range(-45, 45, 5):
        # Detect faces in the image
        faces = faceCascade.detectMultiScale3(
        rotateImage(gray, angle),
        scaleFactor=1.1,
        minNeighbors=10,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE,
        outputRejectLevels = True)

        #neighbours = faces[1]
        rects   = faces[0]
        weights = faces[2]
        if type(weights) == type(np.array([])):
            max_weight = weights.max()
            mask  = weights == max_weight
            rects = rects[mask.reshape(-1)]
            
            _weights.append(max_weight)
            _rects.append(rects)          
            _angles.append(angle)
            
    if _weights:        
        _weights = np.array(_weights)
        _rects   = np.array(_rects)
        _angles  = np.array(_angles)

        max_weight  = _weights.max()
        mask  = _weights == max_weight
        rects = _rects[mask.reshape(-1)]
        angle = _angles[mask.reshape(-1)]
        # Draw a rectangle around the faces
        for x, y, w, h in rects[0]:
#             x, y, w, h = smooth([x, y, w, h], threshold = 5)
            
            lower_h = int((h+y)*1)
        
        
            fullpath = os.path.join(path,'_{}.png'.format(n+count_files))
            
            
            to_save = cv2.resize(image[y:lower_h,x:w+x], (32,32))
            cv2.imwrite(fullpath, to_save)
            cv2.putText(image, "weight:{}, angle:{}".format(max_weight.round(), angle),org=(0,10),fontFace =font,fontScale=0.5,color=(0,255,0))
            cv2.rectangle(image, (x, y), (x+w, lower_h), (0, 255, 0), 2)

    #         roi_gray  = gray[y:y+h, x:x+w]
    #         roi_color = image[y:y+h, x:x+w]
    #         eyes = eyeCascade.detectMultiScale(roi_gray)
    #         for (ex,ey,ew,eh) in eyes:
    #             cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)

    # put the image on screen
    cv2.imshow('Webcam', image)
    time.sleep(1/30)
    if cv2.waitKey(10) & 0xFF == 27:
        break


        
cap.release()        
cv2.destroyAllWindows()

# СКОНВЕРТИРОВАТЬ ДАННЫЕ(фото) В NUMPY МАССИВ ДЛЯ ОБУЧЕНИЯ

In [ ]:
import os 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

path = r"E:\GitHub\udemy\DATA\emoji"
nb_classes = os.listdir(path)


X= []
y= []
range(len(nb_classes))
for n, dir, classes in zip(range(len(nb_classes)), nb_classes, np.eye(len(nb_classes),k=0)):
    fullpath = os.path.join(path, dir)
    print(fullpath)
    for image_name in os.listdir(fullpath):
        image_fullpath = os.path.join(fullpath, image_name)
        
        image = cv2.imread(image_fullpath)
        
        X.append(image)
        y.append([n])
        
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# ОБУЧИТЬ СЕТЬ

In [8]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

# Задаем seed для повторяемости результатов
numpy.random.seed(111)

# Загружаем данные
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 3
# Количество эпох для обучения
nb_epoch = 5#25
# Размер изображений
img_rows, img_cols = 32, 32

# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))

# Слой регуляризации Dropout
model.add(Dropout(0.5))

# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))#softmax , sigmoid

# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Train on 5747 samples, validate on 639 samples
Epoch 1/5
 - 14s - loss: 1.0940 - acc: 0.3698 - val_loss: 1.0578 - val_acc: 0.4773
Epoch 2/5
 - 13s - loss: 0.6724 - acc: 0.6715 - val_loss: 0.3734 - val_acc: 0.8576
Epoch 3/5
 - 14s - loss: 0.3356 - acc: 0.8596 - val_loss: 0.1988 - val_acc: 0.9186
Epoch 4/5
 - 14s - loss: 0.1715 - acc: 0.9382 - val_loss: 0.0897 - val_acc: 0.9718
Epoch 5/5
 - 14s - loss: 0.1084 - acc: 0.9635 - val_loss: 0.0572 - val_acc: 0.9812
Точность работы на тестовых данных: 97.93%


# ИСПОЛЬЗОВАТЬ

In [9]:
import cv2
import sys, os
import itertools
import time
import numpy as np
import scipy
from math import factorial

def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError:
        raise ValueError("window_size and order have to be of type int")
        
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')


def value_to_buffer(value, buffer_name = '', threshold = 5):
    if buffer_name not in globals():
        globals()[buffer_name] = []
          
    globals()[buffer_name].append(value) 
    size = len(globals()[buffer_name])
    
    if size > threshold:
        globals()[buffer_name] = globals()[buffer_name][size-threshold:size]
        

def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


def write_weights(weights, rotate, path):
    fullpath = os.path.join(path, "weights.txt")
    with open(fullpath, 'w') as f:
        f.write("[{}, {}]".format(weights, rotate))

        
        
CIFAR10_LABELS_LIST = [
    '1 - idle', 
    '2 - jaw_open',
    '3 - smile',]

CIFAR10_LABELS_LIST = np.array(CIFAR10_LABELS_LIST)


cap   = cv2.VideoCapture(0)
font  = cv2.FONT_HERSHEY_PLAIN

# Get user supplied values
cascPath = r"E:\GitHub\udemy\DATA\haarcascades"
if not os.path.exists(cascPath):
    cascPath = r"D:\GitHub\udemy\DATA\haarcascades"
# Create the haar cascade

eyeCascade = cv2.CascadeClassifier(os.path.join(cascPath, 'haarcascade_eye.xml'))
faceCascade = cv2.CascadeClassifier(os.path.join(cascPath,'haarcascade_frontalface_default.xml'))



for n in itertools.count():
    ret, image = cap.read()
    image = cv2.resize(image, (160,120))
    # Read the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _weights = []
    _rects   = []
    _angles  = []
    for angle in range(-35, 35, 5):
        # Detect faces in the image
        faces = faceCascade.detectMultiScale3(
        rotateImage(gray, angle),
        scaleFactor=1.1,
        minNeighbors=10,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE,
        outputRejectLevels = True)

        #neighbours = faces[1]
        rects   = faces[0]
        weights = faces[2]
        if type(weights) == type(np.array([])):
            max_weight = weights.max()
            mask  = weights == max_weight
            rects = rects[mask.reshape(-1)]
            
            _weights.append(max_weight)
            _rects.append(rects)          
            _angles.append(angle)
            
    if _weights:        
        _weights = np.array(_weights)
        _rects   = np.array(_rects)
        _angles  = np.array(_angles)

        max_weight  = _weights.max()
        mask  = _weights == max_weight
        rects = _rects[mask.reshape(-1)]
        angle = _angles[mask.reshape(-1)]
        # Draw a rectangle around the faces
        for x, y, w, h in rects[0]:  
            lower_h = int((h+y)*1)
            
            to_match= cv2.resize(image[y:lower_h,x:w+x], (32,32)) /255
            weights = model.predict(np.array([to_match]))[0]
            mask    = weights == weights.max()
            
            
            
            
            value_to_buffer(float(weights[0]), buffer_name = '_idle',    threshold = 6)
            value_to_buffer(float(weights[1]), buffer_name = '_jaw_open',threshold = 6)
            value_to_buffer(float(weights[2]), buffer_name = '_smile',   threshold = 6)
            value_to_buffer(float(angle  [0]), buffer_name = '_rotateZ', threshold = 6)
            
            if len(_idle) >= 6:
                idle     = savitzky_golay(np.array(_idle), window_size = 5, order = 1, deriv=0, rate=1)    [-2]
                jaw_open = savitzky_golay(np.array(_jaw_open), window_size = 5, order = 1, deriv=0, rate=1)[-2]
                smile    = savitzky_golay(np.array(_smile), window_size = 5, order = 1, deriv=0, rate=1)   [-2]
                rotateZ  = savitzky_golay(np.array(_rotateZ), window_size = 5, order = 1, deriv=0, rate=1) [-2]

            
                write_weights([idle,
                              jaw_open,
                              smile], 
                              rotateZ, path = r'E:\GitHub\udemy')###

                cv2.putText(img      = image, 
                            text     = "{},{}".format(CIFAR10_LABELS_LIST[mask], weights.round(3)),
                            org      = (0,10),
                            fontFace = font,
                            fontScale= 0.5,
                            color    = (0,255,0))
                
                
            cv2.rectangle(image, (x, y), (x+w, lower_h), (0, 255, 0), 2)

        #         roi_gray  = gray[y:y+h, x:x+w]
    #         roi_color = image[y:y+h, x:x+w]
    #         eyes = eyeCascade.detectMultiScale(roi_gray)
    #         for (ex,ey,ew,eh) in eyes:
    #             cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)

    # put the image on screen
    cv2.imshow('Webcam', image)
    time.sleep(1/30)
    if cv2.waitKey(10) & 0xFF == 27:
        break

del _idle
del _jaw_open
del _smile
del _rotateZ


cap.release()        
cv2.destroyAllWindows()